## Applied Data Science Capstone Week 4- The Battle of Neighborhoods
#### Project - Selecting the Best location to open an Indian Restaurant in Manhattan, New York


### Import Libraries
* In this section we import the libraries that will be required to process the data.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


### About Dataset

* In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.


* This dataset exists for free on the web. Here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

In [2]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

### Tranforming data into pandas dataframe

In [3]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
# checking our data frame
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
#checking Borough counts
neighborhoods.Borough.value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

In [6]:
# We are interested in Manhattan, having count of 40.

In [7]:
#getting coordinates for new york city
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods of Manhattan superimposed on top.

In [8]:
# creating a new dataframe with neighborhoods only from Manhattan borough
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [9]:
import folium
# map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)
# to add markers
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Getting venues from Foursquare

In [10]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [11]:
#foursquare credentials
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'LJRFN01BD0UIR4AHDDC1MIBNK12SVTAM0U3LEXNMZUVUN1ZO'
CLIENT_SECRET = 'CXQASOEUX1VE2FQN5MTF4WBKY10TLJ2HHTZU1J3T25ARSAIX'
VERSION = '20181020'

In [13]:
#https://developer.foursquare.com/docs/resources/categories
#Indian restaurant = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_indian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d10f941735')
newyork_venues_indian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Riverdale Indian Cuisine,40.880886,-73.908800,Indian Restaurant
1,Marble Hill,40.876551,-73.910660,Tazmohol Indian Restaurant,40.878058,-73.903045,Indian Restaurant
2,Marble Hill,40.876551,-73.910660,Cumin Indian Cuisine,40.886459,-73.909816,Indian Restaurant
3,Chinatown,40.715618,-73.994279,Kabab Bites,40.720094,-73.995819,Indian Restaurant
4,Chinatown,40.715618,-73.994279,indi thai,40.719830,-73.990350,Indian Restaurant


In [14]:
newyork_venues_indian.shape

(1093, 7)

In [15]:
# we got venue details for 1093 Indian restaurant in Manhattan neighborhood

In [16]:
# Plotting these venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.5).add_to(existingMap)

In [17]:
map_newyork_indian = folium.Map(location=[latitude, longitude], zoom_start=11)
addToMap(newyork_venues_indian, 'red', map_newyork_indian)

map_newyork_indian

In [18]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [19]:
# venues grouped by neighborhoods
manhattan_grouped = newyork_venues_indian.groupby('Neighborhood').count()
print(manhattan_grouped.shape)
print()
manhattan_grouped

(39, 6)



,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,26,26,26,26,26,26
Carnegie Hill,12,12,12,12,12,12
Central Harlem,11,11,11,11,11,11
Chelsea,17,17,17,17,17,17
Chinatown,25,25,25,25,25,25
Civic Center,30,30,30,30,30,30
Clinton,46,46,46,46,46,46
East Harlem,6,6,6,6,6,6
East Village,48,48,48,48,48,48


## Analysis of each of 39 neighborhoods

In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_indian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_indian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Court,Food Truck,Himalayan Restaurant,Indian Restaurant,Music Store,North Indian Restaurant,Pakistani Restaurant,South Indian Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
print(manhattan_grouped.shape)
manhattan_grouped

(39, 17)


,Neighborhood,Asian Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Court,Food Truck,Himalayan Restaurant,Indian Restaurant,Music Store,North Indian Restaurant,Pakistani Restaurant,South Indian Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115385,0.000000,0.807692,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.916667,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.823529,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
4,Chinatown,0.000000,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.000000,0.880000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000
5,Civic Center,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.100000,0.000000,0.800000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.108696,0.000000,0.826087,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.020833,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.833333,0.000000,0.020833,0.000000,0.020833,0.000000,0.020833
9,Financial District,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.178571,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant
1,Carnegie Hill,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
2,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
3,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Court
4,Chinatown,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant


### Cluster Neighborhoods

In [24]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 2, 3, 1, 3, 3, 2, 1, 3], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [26]:
manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged= manhattan_merged.dropna()
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
1,Manhattan,Chinatown,40.715618,-73.994279,1.0,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
5,Manhattan,Manhattanville,40.816934,-73.957385,2.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court


In [27]:
manhattan_merged['Cluster Labels']= manhattan_merged['Cluster Labels'].astype(int)
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
1,Manhattan,Chinatown,40.715618,-73.994279,1,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
5,Manhattan,Manhattanville,40.816934,-73.957385,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

### Checking all 5 clusters

In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Gramercy,Indian Restaurant,South Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Dosa Place,Deli / Bodega,Chaat Place,Burrito Place,Asian Restaurant,Tapas Restaurant
37,Stuyvesant Town,Indian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Chaat Place,Tapas Restaurant,Pakistani Restaurant,Music Store


In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant
10,Lenox Hill,Indian Restaurant,North Indian Restaurant,Food Court,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant
11,Roosevelt Island,Indian Restaurant,Food Court,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant
19,East Village,Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,South Indian Restaurant,North Indian Restaurant,Chaat Place,Tapas Restaurant,Pakistani Restaurant,Music Store
20,Lower East Side,Indian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,North Indian Restaurant,Deli / Bodega,Chaat Place,Tapas Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant
21,Tribeca,Indian Restaurant,Food Truck,Tapas Restaurant,Dosa Place,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store
22,Little Italy,Indian Restaurant,Dosa Place,North Indian Restaurant,Deli / Bodega,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store
23,Soho,Indian Restaurant,Food Truck,Dosa Place,North Indian Restaurant,Deli / Bodega,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
31,Noho,Indian Restaurant,Dosa Place,Vegetarian / Vegan Restaurant,South Indian Restaurant,Food Truck,Deli / Bodega,Chaat Place,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant
34,Sutton Place,Indian Restaurant,Food Truck,Food Court,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store


In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
2,Washington Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
4,Hamilton Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
5,Manhattanville,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
6,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
7,East Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
8,Upper East Side,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
9,Yorkville,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
12,Upper West Side,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Truck,Food Court
13,Lincoln Square,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Court


In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Clinton,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant
17,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Court
24,West Village,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant,Food Court
28,Battery Park City,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant
29,Financial District,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant
32,Civic Center,Indian Restaurant,Food Truck,Tapas Restaurant,Dosa Place,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Indian Restaurant,Food Truck,South Indian Restaurant,North Indian Restaurant,Pakistani Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Music Store,Himalayan Restaurant
16,Murray Hill,Indian Restaurant,South Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Pakistani Restaurant,Food Truck,Diner,Tapas Restaurant,Music Store,Himalayan Restaurant
33,Midtown South,Indian Restaurant,South Indian Restaurant,North Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Pakistani Restaurant,Diner,Tapas Restaurant,Music Store,Himalayan Restaurant
38,Flatiron,Indian Restaurant,North Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Dosa Place,Diner,Deli / Bodega,Asian Restaurant,Tapas Restaurant
39,Hudson Yards,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Music Store,Himalayan Restaurant


### These are all our 39 neighborhoods in Manhattan, each clustered into one of the 5 clusters. Check report section for conclusion.